In [1]:
import pandas as pd
import datetime

In [2]:
starttime = datetime.datetime.now()

In [3]:
# 这个文件名和目录会发生变化，记得按实际情况修改
dataDF = pd.read_excel("Data/AudiPotentialCustomer/1月整月潜客量原始_20210203导出.xlsx", sheet_name=0)
setDF = pd.read_excel("Data/AudiPotentialCustomer/车型代码－2021版_update0127.xlsx")

In [4]:
# 把读入的数据列名改成我们处理时的名字，这个也可能会需要按实际情况修改。
dataDF.rename(columns={'快速筛选-日期':'日期', 'Unnamed: 6':'潜客量', '经销商CODE':'经销商代码', '一级渠道':'客户来源_原始', '车系分组':'分组名称'},inplace=True)

In [5]:
# 这里需要增加一个判断，dataDF 前四列不能有空值，潜客量也不能为空。
#      日期 大区名称 小区名称 经销商代码
# 建乐 有时间补充

In [6]:
def datestr(year, mon):
    if len(str(mon)) < 2:
        return str(year)+'0'+str(mon)
    else:
        return str(year)+str(mon)

In [7]:
dataNum = datestr(dataDF.loc[1]['日期'].year, dataDF.loc[1]['日期'].month)
dataNum

'202101'

In [8]:
# 读取配置信息并排重获得将要输出的数据
setDF.rename(columns={'经销商':'经销商代码'}, inplace=True)
hash1 = setDF[['经销商代码', '城市']].drop_duplicates()
hash2 = setDF[['经销商代码', '经销商名称']].drop_duplicates()
hash3 = setDF[['Region', '大区']].drop_duplicates()
hash4 = dataDF[['分组名称', '分组名称']].drop_duplicates()
# 重命名列
hash3.rename(columns={'大区': '大区名称', 'Region': '大区'}, inplace=True)
hash4.columns = ['分组名称', '车型new']

In [9]:
def covTypes(df):
    for col in df.columns:
        if col == '潜客量':
            df[col] = df[col].astype('int32')
        elif col== '日期':
            continue
        else:
            df[col] = df[col].astype('category')

In [10]:
covTypes(dataDF)

In [11]:
# 如果每次都在这里生成pkl，那么missval处理时就不用再来一遍了
# 存一个压缩过的pkl文件，使用bz2格式
# 如果使用压缩，那么在读取的时候也需要增加 compression='bz2'
dataDF.to_pickle("Data/AudiPotentialCustomer/History/potential.%s.pkl" % dataNum, compression='bz2')

In [12]:
# 写excel文件
settingExcel = pd.ExcelWriter("Data/AudiPotentialCustomer/settingALL.xlsx")
hash1.to_excel(settingExcel, sheet_name="city", index=False)
hash2.to_excel(settingExcel, sheet_name="cname", index=False)
hash3[['大区名称', '大区']].to_excel(settingExcel, sheet_name='bigarea', index=False)
hash4.to_excel(settingExcel, sheet_name='cartype', index=False)
settingExcel.save()
settingExcel.close()

In [13]:
endtime = datetime.datetime.now()

In [14]:
(endtime - starttime).seconds

21